In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
p=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        p.append(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install textacy

     |████████████████████████████████| 206 kB 3.3 MB/s 
     |████████████████████████████████| 3.0 MB 8.2 MB/s 
     |████████████████████████████████| 90 kB 5.6 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import spacy
import textacy

# Subject Verb Object detection


class KnowledgeExtraction:

    def retrieveKnowledge(self, textInput):
        nlp = spacy.load('en')
        text = nlp(textInput)
        text_ext = textacy.extract.subject_verb_object_triples(text)
        return list(text_ext)

In [4]:
import requests
import json


class EntityRecognitionLinking:

    class APIError(Exception):

        def __init__(self, status):
            self.status = status

        def __str__(self):
            return "APIError: status={}".format(self.status)

    def entityRecogLink(self, text):

        # Base URL for Spotlight API
        base_url = "http://api.dbpedia-spotlight.org/en/annotate"
        # Parameters
        # 'text' - text to be annotated
        # 'confidence' -   confidence score for linking
        #params = {"text": "My name is Sundar. I am currently doing Master's in Artificial Intelligence at NUS. I love Natural Language Processing.", "confidence": 0.35}
        params = {"text": text, "confidence": 0.35}
        # Response content type
        #headers = {'accept': 'text/html'}
        headers = {'accept': 'application/json'}
        # GET Request
        res = requests.get(base_url, params=params, headers=headers)
        if res.status_code != 200:
            # Something went wrong
            raise APIError(res.status_code)
        # Display the result as HTML in Jupyter Notebook
        # display(HTML(res.text))
        # Pretty printing as json
        print(json.dumps(json.loads(res.text), sort_keys=True, indent=4))
        return json.loads(res.text)

In [5]:
!python -m spacy download en_core_web_sm



You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [6]:
from spacy.matcher import Matcher 
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", None, pattern) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [7]:
import numpy as np
import json
import os
import csv
import re
from nltk.tokenize import sent_tokenize
from collections import defaultdict 
import en_core_web_sm



    
num_files_processed = 0
#sentences_np_array = defaultdict(list)
    

for i in range(8,10):
    pa=p[i]
    index_in_docs = 0
    count=0
        
    if pa.endswith(".json"):
        #sentences_np_array[num_files_processed].append(pa)
    
        
        
        
        with open(pa) as f:
            data = json.load(f)
            
            knowledgeExtractionObj =KnowledgeExtraction()
            
            
            nlp = en_core_web_sm.load()
            col_dict = {}
            seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
            for label, colour in zip(nlp.pipe_labels['ner'], seven_colours):
                col_dict[label] = colour
            options = {'ents': nlp.pipe_labels['ner'], 'colors':col_dict}





                    

                    # Collect abstract sentences
            abstracts = data['abstract']
            for content in abstracts:
                abstract_para = content['text']
                sentences = sent_tokenize(abstract_para)
                index_in_docs += 1
                
                for sentence in sentences:
                    doc1 = nlp(sentence)
                    
                    #sop_list = knowledgeExtractionObj.retrieveKnowledge(sentence)
                    if len(doc1.ents)>=2:
                        spacy.displacy.render(doc1, style='ent', jupyter=True, options=options)
                        print('Subject-verb-object pairs:',knowledgeExtractionObj.retrieveKnowledge(sentence))
                        print('Relation:',get_relation(sentence))
                        #sentences_np_array[num_files_processed].append([sentence,sop_list])
                        
                    count += len(re.findall(r'\w+', sentence))
                    

                    # Collect body sentences
            body_texts = data['body_text']
            for content in body_texts:
                body_para = content['text']
                sentences = sent_tokenize(body_para)
                index_in_docs += 1
                for sentence in sentences:
                    doc1 = nlp(sentence)
                    sop_list = knowledgeExtractionObj.retrieveKnowledge(sentence)
                    if len(doc1.ents)>=2:
                        spacy.displacy.render(doc1, style='ent', jupyter=True, options=options)
                        print('Subject-verb-object pairs:',knowledgeExtractionObj.retrieveKnowledge(sentence))
                        print('Relation:',get_relation(sentence))
                    count += len(re.findall(r'\w+', sentence))
                    #index_in_docs += 1
                    
                    
        num_files_processed+=1
        print('Avg number of abstracts and body texts in '+pa+' dataset: ' + str(index_in_docs))
        print('Avg number of word in abstracts of '+pa+' dataset: ' + str(count//index_in_docs))
        index_in_docs = 0
        count=0
#print(sentences_np_array)
                

                


Subject-verb-object pairs: [(material, to enter, cell)]
Relation: is responsible


Subject-verb-object pairs: [(III proteins, contain, balance)]
Relation: are


Subject-verb-object pairs: [(I proteins, include, influenza), (I proteins, include, immunodeficiency virus), (class II, includes, dengue), (class II, includes, zika), (class II, includes, viruses), (class III, includes, stomatitis virus), (class III, includes, herpesvirus)]
Relation: include


Subject-verb-object pairs: []
Relation: 11


Subject-verb-object pairs: []
Relation: are


...

Subject-verb-object pairs: []
Relation: confirmed in


Subject-verb-object pairs: []
Relation: lived


Subject-verb-object pairs: []
Relation: survived


Subject-verb-object pairs: [(ω, has been demonstrated, have)]
Relation: be


Subject-verb-object pairs: [(ω, been registered, to treat), (it, shows, efficacy)]
Relation: shows comparable
Avg number of abstracts and body texts in /kaggle/input/dataset/CORD-19-research-challenge/custom_license/custom_license/af9d045b63744eb78fa9b2c1ad2a27169d9e0f01.json dataset: 26
Avg number of word in abstracts of /kaggle/input/dataset/CORD-19-research-challenge/custom_license/custom_license/af9d045b63744eb78fa9b2c1ad2a27169d9e0f01.json dataset: 236
